In [78]:

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings

warnings.filterwarnings("ignore")
le = preprocessing.LabelEncoder()

In [20]:
test = pd.read_csv('../dataset/SECTION THREE DATA SET/test_resource_data_q1.csv')
test.head()

,physical_cores,logical_cores,max_cpu_frequency,min_cpu_frequency,total_ram,total_available_ram,total_swap,free_swap,task_count,task_type,current_cpu_frequency
0,20,20,0.00Mhz,0.00Mhz,31.26GB,30.33GB,15.75GB,15.75GB,532,3,2500.00Mhz
1,20,20,0.00Mhz,0.00Mhz,31.26GB,30.29GB,15.75GB,15.75GB,307,8,2500.00Mhz
2,20,20,0.00Mhz,0.00Mhz,31.26GB,30.32GB,15.75GB,15.74GB,4100,5,2500.00Mhz
3,20,20,0.00Mhz,0.00Mhz,31.26GB,30.28GB,15.75GB,15.75GB,1,7,2500.00Mhz
4,20,20,0.00Mhz,0.00Mhz,31.26GB,30.24GB,15.75GB,15.74GB,2133,5,2500.00Mhz


In [37]:
train = pd.read_csv('../dataset/SECTION THREE DATA SET/resource_dataset.csv')
train.head(10)

,physical_cores,logical_cores,max_cpu_frequency,min_cpu_frequency,total_ram,total_available_ram,total_swap,free_swap,start_time,stop_time,time_spend,task_count,task_type,current_cpu_frequency
0,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,1.767967e-05,1.807136e-04,0.000163,532,0,2095.00Mhz
1,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,1.865334e-04,2.271928e-04,0.000041,532,1,2095.00Mhz
2,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,2.291967e-04,2.831338e-04,0.000054,532,2,2095.00Mhz
3,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,1.987467e-05,5.226349e-05,0.000032,532,3,2095.00Mhz
4,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,5.408354e-05,8.176985e-05,0.000028,532,4,2095.00Mhz
5,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.36GB,114.50GB,26.65GB,8.354345e-05,1.116237e-04,0.000028,532,5,2095.00Mhz
6,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.35GB,114.50GB,26.63GB,1.771319e-05,2.851244e-05,0.000011,1,6,2095.00Mhz
7,12,12,2095.00Mhz,0.00Mhz,100.00GB,17.35GB,114.50GB,26.64GB,1.830324e-05,3.979985e-05,0.000021,1,7,2095.00Mhz
8,8,8,0.00Mhz,0.00Mhz,62.76GB,61.48GB,31.50GB,31.50GB,1.620558e+09,1.620558e+09,233.708296,424,4,2095.08Mhz
9,8,8,0.00Mhz,0.00Mhz,62.76GB,61.47GB,31.50GB,31.50GB,1.620558e+09,1.620558e+09,203.794506,424,3,2095.08Mhz


In [38]:
def transformSpeed(x):
    val = 0
    if isinstance(x, float):
        return x
    if 'Mhz' in x:
        val = x.split('Mhz')[0]
    else:
        return x
    return float(val)

def transformMem(x):
    val = 0
    if 'GB' in x:
        val =x.split('GB')[0]
    elif 'MB' in x:
        val = x.split('MB')[0]
    else:
        return x
    return float(val)
    

train.max_cpu_frequency = train.max_cpu_frequency.apply(transformSpeed)
train.min_cpu_frequency = train.min_cpu_frequency.apply(transformSpeed)
train.total_ram = train.total_ram.apply(transformMem)
train.total_available_ram = train.total_available_ram.apply(transformMem)
train.total_swap = train.total_swap.apply(transformMem)
train.free_swap = train.free_swap.apply(transformMem)
train.current_cpu_frequency = train.current_cpu_frequency.apply(transformSpeed)
train.head(10)

,physical_cores,logical_cores,max_cpu_frequency,min_cpu_frequency,total_ram,total_available_ram,total_swap,free_swap,start_time,stop_time,time_spend,task_count,task_type,current_cpu_frequency
0,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,1.767967e-05,1.807136e-04,0.000163,532,0,2095.00
1,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,1.865334e-04,2.271928e-04,0.000041,532,1,2095.00
2,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,2.291967e-04,2.831338e-04,0.000054,532,2,2095.00
3,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,1.987467e-05,5.226349e-05,0.000032,532,3,2095.00
4,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,5.408354e-05,8.176985e-05,0.000028,532,4,2095.00
5,12,12,2095.0,0.0,100.00,17.36,114.5,26.65,8.354345e-05,1.116237e-04,0.000028,532,5,2095.00
6,12,12,2095.0,0.0,100.00,17.35,114.5,26.63,1.771319e-05,2.851244e-05,0.000011,1,6,2095.00
7,12,12,2095.0,0.0,100.00,17.35,114.5,26.64,1.830324e-05,3.979985e-05,0.000021,1,7,2095.00
8,8,8,0.0,0.0,62.76,61.48,31.5,31.50,1.620558e+09,1.620558e+09,233.708296,424,4,2095.08
9,8,8,0.0,0.0,62.76,61.47,31.5,31.50,1.620558e+09,1.620558e+09,203.794506,424,3,2095.08


In [39]:
print(np.any(np.isnan(train)))
print(np.all(np.isfinite(train)))

True
False


In [40]:
train = train.dropna()
print(np.any(np.isnan(train)))

False


In [144]:
predictors = list(test.columns)
print('predictors: ', predictors)

predictors:  ['physical_cores', 'logical_cores', 'max_cpu_frequency', 'min_cpu_frequency', 'total_ram', 'total_available_ram', 'total_swap', 'free_swap', 'task_count', 'task_type', 'current_cpu_frequency']


In [145]:
x_train_df = train[predictors]
x_train_df.head()


,physical_cores,logical_cores,max_cpu_frequency,min_cpu_frequency,total_ram,total_available_ram,total_swap,free_swap,task_count,task_type,current_cpu_frequency
0,12,12,2095.0,0.0,100.0,17.36,114.5,26.65,532,0,2095.0
1,12,12,2095.0,0.0,100.0,17.36,114.5,26.65,532,1,2095.0
2,12,12,2095.0,0.0,100.0,17.36,114.5,26.65,532,2,2095.0
3,12,12,2095.0,0.0,100.0,17.36,114.5,26.65,532,3,2095.0
4,12,12,2095.0,0.0,100.0,17.36,114.5,26.65,532,4,2095.0


In [146]:
y_train_df = train[['time_spend']]
y_train_df.head()

,time_spend
0,0.000163
1,0.000041
2,0.000054
3,0.000032
4,0.000028


In [147]:
# le.fit(y_train_df[['time_spend']])
# y_train_df.time_spend = le.transform(y_train_df[['time_spend']])
# y_train_df.head()

In [148]:
X = np.array(x_train_df)
Y = np.array(y_train_df)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

print('Train size: {}\nTest Size: {}'.format(len(X_train), len(X_test)))

Train size: 7950
Test Size: 1988


In [149]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train)

LinearRegression()

In [150]:
#To retrieve the intercept:
print(regressor.intercept_)

#For retrieving the slope:
print(regressor.coef_)

[782.44368189]
[[ 2.31236108e+02 -2.74835334e+02  3.31594226e-01 -6.10503007e-01
   7.86398998e+00 -3.29982085e-03 -1.53516284e+01  1.22257815e+01
   7.54304436e-02 -3.96654996e+00  1.67977828e-02]]


In [153]:
test.max_cpu_frequency = test.max_cpu_frequency.apply(transformSpeed)
test.min_cpu_frequency = test.min_cpu_frequency.apply(transformSpeed)
test.total_ram = test.total_ram.apply(transformMem)
test.total_available_ram = test.total_available_ram.apply(transformMem)
test.total_swap = test.total_swap.apply(transformMem)
test.free_swap = test.free_swap.apply(transformMem)
test.current_cpu_frequency = test.current_cpu_frequency.apply(transformSpeed)
test.head(10)

,physical_cores,logical_cores,max_cpu_frequency,min_cpu_frequency,total_ram,total_available_ram,total_swap,free_swap,task_count,task_type,current_cpu_frequency
0,20,20,0.0,0.0,31.26,30.33,15.75,15.75,532,3,2500.0
1,20,20,0.0,0.0,31.26,30.29,15.75,15.75,307,8,2500.0
2,20,20,0.0,0.0,31.26,30.32,15.75,15.74,4100,5,2500.0
3,20,20,0.0,0.0,31.26,30.28,15.75,15.75,1,7,2500.0
4,20,20,0.0,0.0,31.26,30.24,15.75,15.74,2133,5,2500.0
5,20,20,0.0,0.0,31.26,30.28,15.75,15.75,3896,5,2500.0
6,20,20,0.0,0.0,31.26,30.34,15.75,15.75,1,6,2500.0
7,20,20,0.0,0.0,31.26,30.19,15.75,15.75,494,3,2500.0
8,20,20,0.0,0.0,31.26,30.32,15.75,15.75,2364,3,2500.0
9,20,20,0.0,0.0,31.26,30.18,15.75,15.75,435,3,2500.0


In [159]:
preds = []
for index, row in test.iterrows():
    pred = regressor.predict([row])
    preds.append(pred)
    
preds

[array([[177.17911655]]),
 array([[140.37464892]]),
 array([[438.25961471]]),
 array([[121.25951612]]),
 array([[289.88819606]]),
 array([[422.99419401]]),
 array([[125.2258681]]),
 array([[174.31322167]]),
 array([[315.36772229]]),
 array([[169.86285849]]),
 array([[315.36785428]]),
 array([[184.42027415]]),
 array([[438.38190552]]),
 array([[315.63456128]]),
 array([[143.31091335]]),
 array([[160.73544483]]),
 array([[-23.26893212]]),
 array([[122.1889275]]),
 array([[299.18157245]]),
 array([[127.99677468]]),
 array([[11.49153306]]),
 array([[141.79485801]]),
 array([[321.8418701]]),
 array([[130.50886154]]),
 array([[268.44267187]]),
 array([[-31.20206504]]),
 array([[127.77783083]]),
 array([[199.99044008]]),
 array([[199.99044008]]),
 array([[199.81410249]]),
 array([[163.44375831]]),
 array([[153.60078324]]),
 array([[206.9160649]]),
 array([[251.50475]]),
 array([[135.37781343]]),
 array([[442.22642865]]),
 array([[289.1180599]]),
 array([[183.36457792]]),
 array([[149.291042]]